In [1]:
%load_ext dotenv
%dotenv
%load_ext autoreload
%autoreload 2

In [2]:
import os 
import pandas as pd
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from df_llm_analyzer import DataFrameLLMAnalyzer
from pydantic import BaseModel, Field
from typing import List, Dict

# PDF Extraction

In [3]:
pdf_list = [
    "/Users/carlosmorales/Desktop/Carlos/personal_projects/afganistan_news/pdf_files/23 Four U.S. Soldiers Killed in Kandahar - The New York Times.pdf"]
articles_content_list = []
articles_id_list = []


for pdf in pdf_list:
    article_text = ""
    loader = PyPDFLoader(pdf)
    pages = loader.load_and_split()
    for page in pages:
        article_text += page.page_content
        article_text += "\n"
    articles_content_list.append(article_text)
    articles_id_list.append(str(hash(article_text)))
    

In [4]:
df_articles = pd.DataFrame({"id": articles_id_list, "text": articles_content_list})
df_articles.head(1)

,id,text
0,-7835225787912804826,20-7-2021 Four U.S. Soldiers Killed in Kandaha...


In [5]:
for article in articles_content_list:
    print ( " - " * 10)
    print(article)
    break

 -  -  -  -  -  -  -  -  -  - 
20-7-2021 Four U.S. Soldiers Killed in Kandahar - The New Y ork T imes
https://www .nytimes.com/2002/04/15/international/four -us-soldiers-killed-in-kandahar .html 1/2https://w ww.nytimes.com/2002/04/15/interna tional/four-us -soldiers -killed-in-kandahar .html
By T erence N eilan
April 15, 2002
Four American soldiers were killed and one was wounded today when rockets they were disposing of exploded near
Kandahar, Afghanistan, defense officials said.
The injured man was evacuated to the medical facility at Kandahar. The dead soldiers' names were being withheld until
their next of kin had been notified.
Lt. Col. Dave Lapan, a liaison between the Pentagon and Central Command in Tampa, Fla., said that the incident was not
related to hostile fire.
The explosion happened at about 11:30 a.m. local time, or 3 a.m. Eastern time, as the soldiers were disposing of 107-
millimeter rockets, Colonel Lapan said.
"It was an accident," added Maj. Ralph Mills, a spokesman

# Source Identifier

In [1]:
source_identifier_template = f"""
**Instructions for Research Assistant**:
 
Your mission is to meticulously identify and extract all sources cited in news articles, focusing on both direct and indirect mentions. Capture every mentioned source, paying close attention to the geographical context of each source when available. It is crucial to include only mentions with identifiable sources. Whenever a quote is given, ensure to cite it every time it occurs, even if a source name is mentioned more than one time. If a mention does not clearly reference a source, please do not include it.
 
**Key Points to Follow**:
 
1. **Extract and Organize Multiple Sources**: Your goal is to diligently read through the article to identify all instances of direct quotes and indirect references. For each source, include:
   - The source or document name.
   - Their title or the nature of the document.
   - The country of origin if mentioned.
   - **A comprehensive excerpt that includes the actual statement or information attributed to the source.**
 
2. **Response Format**: Provide structured responses for each identified source to ensure clarity and completeness. This includes:
   - **Direct Quotes**: Include the full sentence or paragraph where the source directly provides information or opinion.
   - **Indirect References**: Specify the section of text that discusses the findings, opinions, or information attributed to the source, including the context of the mention.
 
3. **Enhanced Examples**:
   - Direct: Dr. Wei, Energy Minister of China, stated, "Renewable energy is our future."
     - Source: Dr. Wei (Energy Minister, China)
     - Excerpt: "Renewable energy is our future."
   - Indirect: According to the 2020 Global Health Report produced in Switzerland, there has been a significant increase in global health awareness.
     - Source: 2020 Global Health Report (Document, Switzerland)
     - Excerpt: "The report highlights a significant increase in global health awareness."
 
Ensure that the excerpts you provide reflect meaningful content that directly relates to each source’s contribution to the article. If a mention in the article does not include direct information or quotes from the source, it should not be documented.
 
**Article Content**:
 >>>
{article}
<<<
"""


NameError: name 'article' is not defined

In [33]:
class SourceIdentifierFunction(BaseModel):
    """
    Detailed information about all the sources found, including the source name and the associated excerpts from the news article.
    """
    source_names: List[str] = Field(..., description="List of source names, which can be a person, country, document, organization, anonymous, or unnamed source, etc.")
    excerpts: List[str] = Field(..., description="List of excerpts from the article where the sources are mentioned. These can be direct quotes or indirect references, ideally with the geographical context of the sources when available.")



In [34]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

convert_pydantic_to_openai_function(SourceIdentifierFunction)


{'name': 'SourceIdentifierFunction',
 'description': 'Detailed information about all the sources found, including the source name and the associated excerpts from the news article.',
 'parameters': {'properties': {'source_names': {'description': 'List of source names, which can be a person, country, document, organization, anonymous, or unnamed source, etc.',
    'items': {'type': 'string'},
    'type': 'array'},
   'excerpts': {'description': 'List of excerpts from the article where the sources are mentioned. These can be direct quotes or indirect references, ideally with the geographical context of the sources when available.',
    'items': {'type': 'string'},
    'type': 'array'}},
  'required': ['source_names', 'excerpts'],
  'type': 'object'}}

In [38]:
analyzer = DataFrameLLMAnalyzer(
        model_name = "gpt-3.5-turbo-0125",
        temperature = 0,
        prompt_template = source_identifier_template,
        analysis_function = SourceIdentifierFunction,
        df_fields = ["text"],
        results_path = "prompt_1_results/source_analyzer_results.parquet",
)

2024-04-06 20:37:57.698 | DEBUG    | df_llm_analyzer:__init__:59 - Initialized LanguageModelDataFrameAnalyzer with model: gpt-3.5-turbo-0125, temperature: 0


In [39]:
df_sources = analyzer.analyze(df=df_articles)

2024-04-06 20:37:59.026 | INFO     | df_llm_analyzer:analyze:71 - Number of rows to process: 1
2024-04-06 20:37:59.027 | INFO     | df_llm_analyzer:_load_existing_results:92 - No existing file found at prompt_1_results/source_analyzer_results.parquet, starting from scratch.
Analyzing batches: 100%|██████████| 1/1 [00:02<00:00,  2.74s/it]
2024-04-06 20:38:01.775 | INFO     | df_llm_analyzer:_analyze_in_batches:135 - Total Tokens: 1247
2024-04-06 20:38:01.775 | INFO     | df_llm_analyzer:_analyze_in_batches:136 - Total Cost: 0.0008115 USD
2024-04-06 20:38:01.776 | INFO     | df_llm_analyzer:_analyze_in_batches:137 - Results saved to prompt_1_results/source_analyzer_results.parquet


In [40]:
df_sources["source_names"].tolist()

[array(['Terence Neilan', 'Lt. Col. Dave Lapan', 'Maj. Ralph Mills',
        'Maj. Bryan Hilferty', 'The Associated Press'], dtype=object)]

In [41]:
df_sources["excerpts"].tolist()

[array(['Four American soldiers were killed and one was wounded today when rockets they were disposing of exploded near Kandahar, Afghanistan, defense officials said.',
        'Lt. Col. Dave Lapan, a liaison between the Pentagon and Central Command in Tampa, Fla., said that the incident was not related to hostile fire.',
        '"It was an accident," added Maj. Ralph Mills, a spokesman at Central Command.',
        'On Sunday two explosions occurred at an airfield used by coalition forces in the southeastern city of Khost, near the border with Pakistan, Maj. Bryan Hilferty, a United States military spokesman, said.',
        'The explosions came near the spot where two rocket-propelled grenades exploded on Saturday night, he was quoted by The Associated Press as saying.'],
       dtype=object)]

In [31]:
for sources, excerpts in zip(df_sources["source_names"], df_sources["excerpts"]):
    for source, excerpt in zip(sources, excerpts):
        print(f"Source: {source}\nExcerpt: {excerpt}\n{'-'*50}")


Source: Lt. Col. Dave Lapan
Excerpt: Lt. Col. Dave Lapan, a liaison between the Pentagon and Central Command in Tampa, Fla.
--------------------------------------------------
Source: Maj. Ralph Mills
Excerpt: Maj. Ralph Mills, a spokesman at Central Command.
--------------------------------------------------
Source: Maj. Bryan Hilferty
Excerpt: Maj. Bryan Hilferty, a United States military spokesman.
--------------------------------------------------
Source: The Associated Press
Excerpt: he was quoted by The Associated Press as saying.
--------------------------------------------------
